In [3]:
#数据集读取，数据储存于data，标签储存于labels

import numpy as np
import gzip
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist
%matplotlib inline
from imageio import imread,imsave

# 训练集文件
train_images_idx3_ubyte_file = 'C:/Users/HUAWEI/Desktop/minst数据集/train-images-idx3-ubyte.gz'
# 训练集标签文件
train_labels_idx1_ubyte_file = 'C:/Users/HUAWEI/Desktop/minst数据集/train-labels-idx1-ubyte.gz'
# 测试集文件
test_images_idx3_ubyte_file = 'C:/Users/HUAWEI/Desktop/minst数据集/t10k-images-idx3-ubyte.gz'
# 测试集标签文件
test_labels_idx1_ubyte_file = 'C:/Users/HUAWEI/Desktop/minst数据集/t10k-labels-idx1-ubyte.gz'

#提取训练集数据矩阵
f = gzip.open(train_images_idx3_ubyte_file,'r')
image_size = 28 #不能改，不要改！
num_images = 5000 #调整读取数量，可以改
f.read(16) #不能改，不要改！
buf = f.read(image_size * image_size * num_images)
data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
data = data.reshape(num_images, image_size, image_size)

#提取训练集数据标签
f = gzip.open(train_labels_idx1_ubyte_file,'r')
f.read(8) #不能改，不要改！
labels = []
for i in range(0,5000):   
    buf = f.read(1)
    labels.append(np.frombuffer(buf, dtype=np.uint8).astype(np.int64))

print(data.shape)
print(len(labels))

(5000, 28, 28)
5000


In [30]:
num_data = 20
num_client = 20
index = 0
for i in range(num_client):
    #print(data[index:index+num_data])
    exec("client_data_%i = data[index:index+num_data]"%i)
    exec("client_label_%i = labels[index:index+num_data]"%i)
    index += num_data
    
print(client_data_1)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [31]:
def as_row_matrix (X):
    if len (X) == 0:
        return np. array ([])
    mat = np. empty ((0 , X [0].size ), dtype =X [0]. dtype )
    for row in X:
        mat = np.vstack(( mat , np.asarray( row ).reshape(1 , -1))) # 1 x r*c 
    return mat

In [192]:
def get_number_of_components_to_preserve_variance(eigenvalues, variance=.70):
    for ii, eigen_value_cumsum in enumerate(np.cumsum(eigenvalues) / np.sum(eigenvalues)):
        if eigen_value_cumsum > variance:
            return ii

In [198]:
def client_P (X,branch=0 , num_components =0):
    [n,d] = X.shape
    if ( num_components <= 0) or ( num_components >n):
        num_components = n
        mu = X.mean( axis =0)
        X = X - mu
    #if n>d:
        C = np.dot(X.T,X) # Covariance Matrix
    #else :
        #C = np.dot (X,X.T) # Covariance Matrix
        #branch = 1
    return C,branch

In [190]:
def client_AP(X):
    [n,d] = X.shape
    if n>d:
        C = np.dot(X.T,X) # Covariance Matrix
    else :
        C = np.dot (X,X.T) # Covariance Matrix
        [ eigenvalues , eigenvectors ] = np.linalg.eigh(C)
        eigenvectors = np.dot(X.T, eigenvectors )
    for i in range (n):
        eigenvectors [:,i] = eigenvectors [:,i]/ np.linalg.norm( eigenvectors [:,i])
    eigenvalues = np.diag(eigenvalues)   
    return eigenvectors, eigenvalues

In [122]:
def P_COV(X,branch=0,num_components=0):
    matrix=0
    for i in X:
        matrix+=i
    [ eigenvalues , eigenvectors ] = np.linalg.eigh(matrix)
    #print(eigenvectors.shape)
    #if branch == 1:
        #eigenvectors = np.dot(X.T,eigenvectors)
    [n,d] = X[0].shape
    for i in range (n):
        eigenvectors [:,i] = eigenvectors [:,i]/ np.linalg.norm( eigenvectors [:,i])
    idx = np.argsort (- eigenvalues )
    eigenvalues = eigenvalues [idx ]
    eigenvectors = eigenvectors [:, idx ]
    num_components = get_number_of_components_to_preserve_variance(eigenvalues)
    eigenvalues = eigenvalues [0: num_components ].copy ()
    eigenvectors = eigenvectors [: ,0: num_components ].copy ()
    return [ eigenvalues , eigenvectors ]

In [195]:
def AP_Stack(X):
    stack = np. empty ( X[0].size , dtype =X[0]. dtype )
    for i in X:
        stack = np.vstack((stack,i))
    [ eigenvalues , eigenvectors ] = np.linalg.eigh(stack)
    for i in range (n):
        eigenvectors [:,i] = eigenvectors [:,i]/ np.linalg.norm( eigenvectors [:,i])
    idx = np.argsort (- eigenvalues )
    eigenvalues = eigenvalues [idx]
    eigenvectors = eigenvectors [:, idx ]
    num_components = get_number_of_components_to_preserve_variance(eigenvalues)
    eigenvalues = eigenvalues [0: num_components ].copy ()
    eigenvectors = eigenvectors [: ,0: num_components ].copy ()
    return [ eigenvalues , eigenvectors ]

In [187]:
def AP_COV (client_lst):
    M = 0
    for i in range(num_client):
        M += np.dot(client_lst[i][0],np.dot(client_lst[i][1],client_lst[i][0].T))
    [ eigenvalues , eigenvectors ] = np.linalg.eigh(M)
    
    [n,d] = M.shape
    for i in range (n):
        eigenvectors [:,i] = eigenvectors [:,i]/ np.linalg.norm( eigenvectors [:,i])
    idx = np.argsort (- eigenvalues )
    eigenvalues = eigenvalues [idx ]
    eigenvectors = eigenvectors [:, idx ]
    num_components = get_number_of_components_to_preserve_variance(eigenvalues)
    print(num_components)
    eigenvalues = eigenvalues [0: num_components ].copy ()
    eigenvectors = eigenvectors [: ,0: num_components ].copy ()
    print(eigenvalues)
    print(eigenvectors)
    
    return [eigenvalues,eigenvectors]

In [199]:
# P-COV
client_cov = []
branch = 0
for i in range(num_client):
    exec("row_matrix = as_row_matrix(client_data_%i)"%i)
    [cov,branch] = client_P(row_matrix)
    client_cov.append(cov)
    #print(branch)
    #print(row_matrix.shape)
[fed_eigenvalues,fed_eigenvectors] = P_COV(client_cov,branch)
print(fed_eigenvalues)
print(fed_eigenvectors.shape)
print(fed_eigenvectors)

[1.33330984e+08 1.02580304e+08 8.38019200e+07 7.88522320e+07
 6.63114880e+07 6.28882680e+07 4.28036400e+07 3.90679040e+07
 3.56034280e+07 3.01858480e+07 2.79200720e+07 2.44467540e+07
 2.23803920e+07 2.08221860e+07 1.97996700e+07 1.94322760e+07
 1.78099920e+07 1.64027400e+07 1.46783910e+07 1.42240200e+07]
(784, 20)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [196]:
#AP-Stack
client_subspace = []
branch = 0
for i in range(num_client):
    exec("row_matrix = as_row_matrix(client_data_%i)"%i)
    [vec,sig] = client_AP(row_matrix)
    client_subspace.append(np.dot(sig,vec.T))
    #print(branch)
    #print(row_matrix.shape)
[fed_eigenvalues,fed_eigenvectors] = AP_Stack(client_subspace)
print(fed_eigenvectors.shape)
print(fed_eigenvectors)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 15680 and the array at index 1 has size 784

In [193]:
# AP-COV
client_lst = []
for i in range(num_client):
    exec("row_matrix = as_row_matrix(client_data_%i)"%i)
    [vec,sig] = client_AP(row_matrix)
    client_lst.append([vec,sig])

[fed_eigenvalues,fed_eigenvectors] = AP_COV(client_lst)

8
[9.5070086e+08 1.1993904e+08 1.0776582e+08 8.8064288e+07 7.3485024e+07
 6.7090100e+07 5.6849232e+07 4.4001344e+07]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [101]:
def pca (X, num_components =0): 
    [n,d] = X.shape
    if ( num_components <= 0) or ( num_components >n):
        num_components = n
        mu = X.mean( axis =0)
        X = X - mu
    if n>d:
        C = np.dot(X.T,X)
        [ eigenvalues , eigenvectors ] = np.linalg.eigh(C)
    else :
        C = np.dot (X,X.T) 
        [ eigenvalues , eigenvectors ] = np.linalg.eigh(C)
        eigenvectors = np.dot(X.T, eigenvectors )
    for i in range (n):
        eigenvectors [:,i] = eigenvectors [:,i]/ np.linalg.norm( eigenvectors [:,i])
    idx = np.argsort (- eigenvalues )
    eigenvalues = eigenvalues [idx ]
    eigenvectors = eigenvectors [:, idx ]
    num_components = get_number_of_components_to_preserve_variance(eigenvalues)
    print(num_components)
    eigenvalues = eigenvalues [0: num_components ].copy ()
    eigenvectors = eigenvectors [: ,0: num_components ].copy ()
    print(eigenvectors.shape)
    return [ eigenvalues , eigenvectors , mu]
total_data = data[:num_data*num_client]
big_data = as_row_matrix(total_data)
[pca_eigenvalues, pca_eigenvectors, pca_mu] = pca (big_data)  ; #原始PCA，需要总数据大矩阵
print(pca_eigenvectors)

20
(784, 20)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [149]:
difference = []
angle = []
for i in range(20):
    fed = fed_eigenvectors[:,i]
    pca = pca_eigenvectors[:,i]
    pca = pca.T
    diff = np.dot(pca,fed)
    difference.append(diff)
    sin = 1-diff**2
    angle_diff = np.arccos(diff)*180/np.pi
    print("%.5f   %.2f\n"%(sin,angle_diff))
    angle.append(angle_diff)
#print(difference)

ValueError: shapes (784,) and (20,) not aligned: 784 (dim 0) != 20 (dim 0)